In [5]:
import numpy as np
import math
from matplotlib import pyplot as plt
from openmm import app
import openmm as mm
from openmm import unit
import sys
import os

In [2]:
def reload(fname,T = 50, gamma = 1/50, length = 1, dt = 0.1, skipSteps = 1, 
           f_pdb = "pdb", f_ff = "forcefield", srSize = 20):
    #fname = "{}fs_{}ps_{}K_{}ss.xyz".format(dt, length, T, skipSteps)
    totSteps = int(length*1000/(skipSteps * dt))
    print("totSteps", totSteps)

    a = np.loadtxt(fname)
    nParticles = a.shape[-1]//3
    a = a.reshape(totSteps, 3, nParticles)
    print(a.shape)

    return a


In [3]:
def euclidean_distance(r1,r2):
    d = np.sqrt(
        np.sum(
            np.square(r1-r2)
        )
    )
    return d

def find_closest_monomer(p_small,p_large):
    n_small = np.shape(p_small)[0]
    n_large = np.shape(p_large)[0]

    distance_matrix = np.zeros((n_small,n_large))

    # Calculate the distance from each unit of the smaller ring to each unit on 
    # the longer ring
    for i in range(n_small):
        for j in range(n_large):
            r1 = p_small[i,:]
            r2 = p_large[j,:]
            distance_matrix[i,j]= euclidean_distance(r1,r2)

    #Return the distance 
    closest_monomer_to_each_bead = np.argmin(distance_matrix,axis=1)
    return closest_monomer_to_each_bead

def calculate_circle_center_of_mass(ring):
    return np.mean(ring,axis=0)

def calculate_vector_magnitude(v):
    return np.sqrt(np.sum(np.square(v)))

def calculate_angular_displacement():
   
    position_time_tensor =reload("animate_bad.xyz")

    small_ring_carbon_indices=(0,20)
    large_ring_carbon_indices=(20,220)
    
    #Extract the small ring positions
    p_small = position_time_tensor[:,:,
                    small_ring_carbon_indices[0]:small_ring_carbon_indices[1]]
        
    #Extract the large ring positions
    p_large = position_time_tensor[:,:,
                    large_ring_carbon_indices[0]:large_ring_carbon_indices[1]]
    
    # Find the closest monomer to each bead for the initial position, 
    # transposing just to give the matrix the right shape 
    initial_closest_monomers_to_each_bead = find_closest_monomer(
        p_small[0,:,:].T,
        p_large[0,:,:].T
    )
    
    initial_index = int(np.mean(initial_closest_monomers_to_each_bead))
    
    nt = np.shape(position_time_tensor)[0]
    angular_displacement = np.zeros(nt)    

    # find the initial radius of the beads
    theta = 2*np.pi/p_large.shape[-1]
    
    for i in range(nt):
        # find the angle within the plane of r_s_CoM and r_l_CoM for each node
        small_center = calculate_circle_center_of_mass(p_small[i,:,:].T)

        # find the current monomer index
        sm_mono_index = find_closest_monomer(np.array([small_center]), 
                                             p_large[i, :, :].T)
        
        # difference in index between init and current 
        delta = sm_mono_index - initial_index

        # angular distance according to the initial monomer geometry
        ang_disp = delta * theta

        angular_displacement[i] = ang_disp
        
    return angular_displacement

In [ ]:
def runSim(T = 50, gamma = 1/50, length = 10, dt = 0.01, skipSteps = 1, f_pdb = "pdb", 
           f_ff = "forcefield", srSize = 20):
    """
    Function to run the openMM simulation (NVT) with given params and 
    pdb+forcefields

    Saves a xyz file of a compressed 3d array. See the function reload() to get 
    an idea of how to load and manipulate the output array (and format the file 
    name). The final array should have dims defined by 
    [n_timesteps, 3 (xyz), n_particles]

    parameters:
    T: int
        Temperature
    gamma: float
        Friction parameters for Langevin
    len: float
        Total length of the simulation in ps
    dt: float
        Size of the timesteps in ps
    f_pdb: str
        Name of the pdb file (excluding the file extension, which should 
        simply be .pdb)
    f_ff: str
        Name of the forcefield file (excluding the file extension, which should 
        simply be .xml)
    srSize: int
        Number of monomer units in the small ring
    """
    # set up strings
    # THE FOLLOWING FILE NAMES ARE PLACEHOLDERS: CHANGE WHEN POSSIBLE
    path = os.getcwd()
    pdbPath = os.path.join(path, "pdb_files/{}.pdb".format(f_pdb))
    forcefieldPath = os.path.join(path, "pdb_files/{}.xml".format(f_ff))

    # set up params
    kB = 1.38e-23
    beta = 1 # needs kb
    steps = int(length*1000/dt)
    indSteps = int(length*1000/(dt*skipSteps))

    # load external files for topo and PES
    pdb = app.PDBFile(pdbPath)
    topo = pdb.topology

    # identify atoms in topology
    particles = []
    # future update maybe: collect residues to identify number of atoms/chain
    # residues = []
    for atom in topo.atoms():
        particles.append(atom)
        # residues.append(atom.residue)

    # add bonds between the individual rings
    c1 = particles[0:srSize]
    c2 = particles[srSize:]
    numParticles = len(particles)
    # hy1 = particles[20:40]
    # hy2 = particles[40:60]

    # create arrays to store xyz coords in
    # format is timestep:xyz:particle#
    traj = np.zeros((indSteps, 3, numParticles))

    for i, C in enumerate(c1):
        topo.addBond(C, c1[i-1])
        # topo.addBond(C, hy1[i])
        # topo.addBond(C, hy2[i])
    
    for i, C in enumerate(c2):
        topo.addBond(C, c2[i-1])
        # topo.addBond(C, hy1[i])
        # topo.addBond(C, hy2[i])

    forcefield = app.ForceField(forcefieldPath)
    unmatched_residues = forcefield.getUnmatchedResidues(topo)
    print("unmatched residues\n", unmatched_residues)
    nonbonded = app.NoCutoff
         
    system = forcefield.createSystem(topo, nonbondedMethod = nonbonded,
                                    nonbondedCutoff = 1e3*unit.nanometer,
                                    constraints = None)

    integrator = mm.LangevinIntegrator(T*unit.kelvin, gamma, 
                                       dt*unit.femtoseconds)

    platform = mm.Platform.getPlatformByName('Reference')
    simulation = app.Simulation(pdb.topology, system, integrator, platform)
    simulation.context.setPositions(pdb.positions)
    simulation.context.computeVirtualSites()
    simulation.context.setVelocitiesToTemperature(T*unit.kelvin)

    # equilibration step
    # simulation.step(1000)

    for step in range(steps):
        simulation.step(skipSteps)
        # extract positions
        current = simulation.context.getState(getPositions = True)
        positions = np.array(current.getPositions()/unit.nanometer)
        traj[step, :, :] = positions.transpose()

    # save the trajectory as an xyz file
    traj_r = traj.reshape(traj.shape[0], -1)
    print(traj_r.shape)
    np.savetxt("{}fs_{}ps_{}K_{}ss.xyz".format(dt, length, T, skipSteps), traj_r)
    return traj_r

In [4]:
traj_r = runSim(
    T = 1, 
    gamma = 1/50, 
    length = 2, 
    dt = 0.01, 
    skipSteps = 1, 
    f_pdb = "pdb", 
           f_ff = "forcefield", srSize = 20)

totSteps 10000
(10000, 3, 220)


In [ ]:
angs = calculate_angular_displacement()
iters = np.arange(angs.shape[0])
print("shape of angles:", angs.shape)
print("shape of iters:", iters.shape)
plt.plot(iters, angs)
plt.show()
plt.close()